In [4]:
import pandas as pd
import psycopg2
import numpy as np

In [5]:
# access the postgresql server
conn = psycopg2.connect(
    host="codd04.research.northwestern.edu",
    port = "5433",
    database="postgres",
    user="cpdbstudent",
    password="DataSci4AI")
cursor = conn.cursor()


query = "select  officerdata.District_Number_Name as District_Number_Name,\
           officerdata.officer_id as officerdata_id,\
           officerdata.officer_first_name as officer_first_name,\
           officerdata.officer_last_name  as officer_last_name,\
           officerdata.officer_race as officer_race,\
           officerdata.officer_rank as officer_rank,\
           da.officer_id as awardee,\
           da.requester_full_name as requester_full_name\
 from data_award as da join (\
    select\
       unit_description as District_Number_Name,\
       dof_id as officer_id,\
       dof_first_name as officer_first_name,\
       dof_last_name as officer_last_name,\
       dof_race as officer_race,\
       dof_rank as officer_rank\
 from (\
    select\
           dof.id as dof_id,\
           dof.last_name as dof_last_name,\
           dof.first_name as dof_first_name,\
           dof.race as dof_race,\
           dof.rank as dof_rank,\
 area_beat.unit_description\
    from data_officer as dof join (\
        select\
               *, da.id as da_id, dpb.id as dpb_id, area_id as da_area_id\
        from\
             data_policebeat as dpb\
                 join data_area da\
                     on dpb.area_id = da.id) as area_beat\
        on dof.last_unit_id = area_beat.unit_id\
        where unit_description LIKE '%DISTRICT - %'\
    group by dof_id, unit_description, major_award_count) as officer_unitdesc_award) as officerdata\
 on da.officer_id = officerdata.officer_id"
cursor.execute(query)
#where unit_description in ('4TH DISTRICT - SOUTH CHICAGO','10TH DISTRICT - OGDEN')\
officer_data = cursor.fetchall()
df_officer_data = pd.DataFrame(officer_data)
colnames = [desc[0] for desc in cursor.description]
df_officer_data.columns = colnames

query = "select officer_id,requester_full_name from data_award"
cursor.execute(query)

req_name = cursor.fetchall()
df_req_name = pd.DataFrame(req_name)
colnames = [desc[0] for desc in cursor.description]
df_req_name.columns = colnames

In [6]:
policerace = pd.read_excel('policerace.xlsx')
awardee_race = df_officer_data.groupby(['district_number_name','officer_race'])['awardee'].count().reset_index()
district_total_awards = awardee_race.groupby('district_number_name')['awardee'].sum()
awardee_race['percent_by_dist'] = 0
for i in range(len(awardee_race)):
  district = awardee_race['district_number_name'][i]
  
  awardee_race['percent_by_dist'][i] = float((awardee_race['awardee'][i]/ district_total_awards[district])*100)

awardee_race['population_percentage'] = 0.000
for i in range(len(awardee_race)):
 dist = awardee_race['district_number_name'][i]
 race = awardee_race['officer_race'][i] 
 
 try: 
  #print(dist ,'   ',  policerace[policerace['District_Number_Name']==dist][policerace['Race']==race]['Percentage'].values[0])
  awardee_race['population_percentage'][i] = policerace[policerace['District_Number_Name']==dist][policerace['Race']==race]['Percentage'].values[0]
 except:
   awardee_race['population_percentage'][i] = np.nan

awardee_race['population_percentage'] = awardee_race['population_percentage']*100
awardee_race['difference'] = awardee_race['percent_by_dist']/awardee_race['population_percentage']
awardee_race['awards_disproportion'] = awardee_race['difference']-1
for i in range(len(awardee_race['district_number_name'])):
  awardee_race['district_number_name'][i] = awardee_race['district_number_name'][i].split('-')[1].title().strip()
awardee_race.to_csv('awardeerace.csv')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#